In [ ]:
# =============================================================================
# IMPORTS
# =============================================================================

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("✓ Imports loaded")

In [ ]:
# =============================================================================
# DATA LOADING
# =============================================================================

BASKETS = {
    'cybersecurity': ['CRWD', 'PANW', 'ZS', 'FTNT', 'OKTA', 'S', 'VRNS', 'TENB', 'CYBR', 'NET'],
    'uranium': ['UUUU', 'UEC', 'CCJ', 'LEU', 'DNN', 'NXE', 'URG', 'SMR'],
    'quantum': ['IONQ', 'RGTI', 'QBTS'],
    'space': ['RKLB', 'ASTS', 'LUNR'],
}

START_DATE = '2024-01-01'
END_DATE = '2026-01-08'

def load_basket_data(tickers, start=START_DATE, end=END_DATE):
    data = {}
    for ticker in tickers:
        try:
            df = yf.download(ticker, start=start, end=end, progress=False)
            if len(df) > 0:
                df['Returns'] = df['Adj Close'].pct_change()
                df['Ticker'] = ticker
                data[ticker] = df
                print(f"✓ {ticker}: {len(df)} days")
        except Exception as e:
            print(f"✗ {ticker}: {e}")
    return data

def load_sector(sector_name):
    if sector_name in BASKETS:
        print(f"\n📦 Loading {sector_name.upper()}")
        return load_basket_data(BASKETS[sector_name])
    return {}

print("✓ Functions ready")

In [ ]:
# =============================================================================
# CONFIGURATION - DEFINE YOUR SEQUENCE HERE
# =============================================================================

# SEQUENCE HYPOTHESIS:
# "When CRWD moves > 3% on Day 0, and ZS confirms with > 2% on Day 1,
#  then S moves > 3% within the next 3 days"

SEQUENCE = {
    'name': 'CRWD leads ZS leads S',
    'step_1': {
        'ticker': 'CRWD',
        'condition': 'return > 0.03',  # 3% up
        'day': 0
    },
    'step_2': {
        'ticker': 'ZS', 
        'condition': 'return > 0.02',  # 2% up
        'day': 1  # day after step 1
    },
    'target': {
        'ticker': 'S',
        'measure': 'max_return',  # track max return over window
        'window': 3  # days after sequence completes
    }
}

print("✓ Sequence configured")
print(f"  Testing: {SEQUENCE['name']}")
print(f"  Step 1: {SEQUENCE['step_1']['ticker']} {SEQUENCE['step_1']['condition']}")
print(f"  Step 2: {SEQUENCE['step_2']['ticker']} {SEQUENCE['step_2']['condition']} (Day {SEQUENCE['step_2']['day']})")
print(f"  Target: {SEQUENCE['target']['ticker']} {SEQUENCE['target']['measure']} over {SEQUENCE['target']['window']} days")

In [ ]:
# =============================================================================
# SEQUENCE EVALUATION
# =============================================================================

def evaluate_condition(returns, condition):
    """
    Evaluate a condition string against returns.
    """
    # Parse condition
    if '>' in condition:
        parts = condition.split('>')
        threshold = float(parts[1].strip())
        return returns > threshold
    elif '<' in condition:
        parts = condition.split('<')
        threshold = float(parts[1].strip())
        return returns < threshold
    return False

def find_sequences(data, sequence):
    """
    Find all instances where the sequence pattern occurred.
    """
    step1 = sequence['step_1']
    step2 = sequence['step_2']
    target = sequence['target']
    
    ticker1_df = data.get(step1['ticker'])
    ticker2_df = data.get(step2['ticker'])
    target_df = data.get(target['ticker'])
    
    if ticker1_df is None or ticker2_df is None or target_df is None:
        print("✗ Missing data for one or more tickers")
        return pd.DataFrame()
    
    results = []
    
    # Find step 1 triggers
    step1_triggers = ticker1_df[
        evaluate_condition(ticker1_df['Returns'], step1['condition'])
    ]
    
    print(f"\n🔍 Found {len(step1_triggers)} Step 1 triggers")
    
    for date, row in step1_triggers.iterrows():
        # Check step 2 on the specified day offset
        step2_date = date + pd.Timedelta(days=step2['day'])
        
        # Find nearest trading day
        if step2_date not in ticker2_df.index:
            # Look for next available date within 3 days
            for i in range(1, 4):
                check_date = date + pd.Timedelta(days=step2['day'] + i)
                if check_date in ticker2_df.index:
                    step2_date = check_date
                    break
            else:
                continue
        
        if step2_date in ticker2_df.index:
            step2_return = ticker2_df.loc[step2_date, 'Returns']
            
            if evaluate_condition(pd.Series([step2_return]), step2['condition']).iloc[0]:
                # SEQUENCE CONFIRMED! Now track target
                
                target_window_start = step2_date
                idx = target_df.index.get_loc(target_window_start)
                
                if idx + target['window'] < len(target_df):
                    window_data = target_df.iloc[idx:idx + target['window'] + 1]
                    
                    entry_price = window_data.iloc[0]['Adj Close']
                    
                    if target['measure'] == 'max_return':
                        max_price = window_data['Adj Close'].max()
                        target_return = (max_price - entry_price) / entry_price
                    elif target['measure'] == 'end_return':
                        end_price = window_data.iloc[-1]['Adj Close']
                        target_return = (end_price - entry_price) / entry_price
                    
                    results.append({
                        'step1_date': date,
                        'step1_return': row['Returns'],
                        'step2_date': step2_date,
                        'step2_return': step2_return,
                        'target_return': target_return,
                        'win': target_return > 0.02  # target > 2% = win
                    })
    
    return pd.DataFrame(results)

print("✓ Sequence finder ready")

In [ ]:
# =============================================================================
# EXECUTE - TEST THE SEQUENCE
# =============================================================================

print("\n🐺 STARTING SEQUENCE TEST")
print("="*60)

# Load data
all_data = load_sector('cybersecurity')

# Find sequences
sequence_results = find_sequences(all_data, SEQUENCE)

if len(sequence_results) > 0:
    print(f"\n{'='*60}")
    print(f"📊 SEQUENCE TEST RESULTS")
    print(f"{'='*60}")
    print(f"Sequence: {SEQUENCE['step_1']['ticker']} {SEQUENCE['step_1']['condition']}")
    print(f"     Then: {SEQUENCE['step_2']['ticker']} {SEQUENCE['step_2']['condition']}")
    print(f"   Target: {SEQUENCE['target']['ticker']} over {SEQUENCE['target']['window']} days")
    print(f"{'='*60}")
    print(f"Total occurrences: {len(sequence_results)}")
    print(f"Win rate (target > 2%): {sequence_results['win'].mean()*100:.1f}%")
    print(f"Average target return: {sequence_results['target_return'].mean()*100:.2f}%")
    print(f"Max target return: {sequence_results['target_return'].max()*100:.2f}%")
    print(f"Min target return: {sequence_results['target_return'].min()*100:.2f}%")
    
    print(f"\n📋 Individual occurrences:")
    display_df = sequence_results.copy()
    display_df['step1_return'] = display_df['step1_return'].apply(lambda x: f"{x*100:.2f}%")
    display_df['step2_return'] = display_df['step2_return'].apply(lambda x: f"{x*100:.2f}%")
    display_df['target_return'] = display_df['target_return'].apply(lambda x: f"{x*100:.2f}%")
    print(display_df.to_string(index=False))
else:
    print("\n✗ No sequences found matching the pattern")
    print("   Try adjusting thresholds or checking if tickers moved together")

In [ ]:
# =============================================================================
# TEST MULTIPLE SEQUENCES AT ONCE
# =============================================================================

# Easy way to test multiple sequences
SEQUENCES_TO_TEST = [
    {
        'name': 'CRWD leads ZS leads S',
        'step_1': {'ticker': 'CRWD', 'condition': 'return > 0.03', 'day': 0},
        'step_2': {'ticker': 'ZS', 'condition': 'return > 0.02', 'day': 1},
        'target': {'ticker': 'S', 'measure': 'max_return', 'window': 3}
    },
    {
        'name': 'CRWD big move, VRNS follows',
        'step_1': {'ticker': 'CRWD', 'condition': 'return > 0.04', 'day': 0},
        'step_2': {'ticker': 'PANW', 'condition': 'return > 0.01', 'day': 0},  # same day
        'target': {'ticker': 'VRNS', 'measure': 'max_return', 'window': 5}
    },
    {
        'name': 'ZS leads OKTA',
        'step_1': {'ticker': 'ZS', 'condition': 'return > 0.03', 'day': 0},
        'step_2': {'ticker': 'CRWD', 'condition': 'return > 0.02', 'day': 0},
        'target': {'ticker': 'OKTA', 'measure': 'max_return', 'window': 3}
    },
]

print("\n" + "="*60)
print("🔬 TESTING MULTIPLE SEQUENCES")
print("="*60)

multi_results = []

for seq in SEQUENCES_TO_TEST:
    results = find_sequences(all_data, seq)
    if len(results) > 0:
        print(f"\n{seq['name']}:")
        print(f"  Occurrences: {len(results)}")
        print(f"  Win rate: {results['win'].mean()*100:.1f}%")
        print(f"  Avg return: {results['target_return'].mean()*100:.2f}%")
        
        results['sequence_name'] = seq['name']
        multi_results.append(results)
    else:
        print(f"\n{seq['name']}: No matches found")

# Combine all results
if multi_results:
    all_seq_results = pd.concat(multi_results, ignore_index=True)
    
    print("\n" + "="*60)
    print("📊 SUMMARY OF ALL SEQUENCES")
    print("="*60)
    
    summary = all_seq_results.groupby('sequence_name').agg({
        'target_return': ['mean', 'count'],
        'win': 'mean'
    }).round(4)
    
    summary.columns = ['avg_return', 'occurrences', 'win_rate']
    summary = summary.reset_index()
    summary['expected_value'] = summary['avg_return'] * summary['win_rate']
    
    print(summary.sort_values('expected_value', ascending=False).to_string(index=False))

---

## 🎯 HOW TO USE THIS NOTEBOOK

### Testing Your Own Sequence

1. **Edit the SEQUENCE configuration cell:**
   ```python
   SEQUENCE = {
       'name': 'Your sequence name',
       'step_1': {
           'ticker': 'TICKER1',
           'condition': 'return > 0.03',  # 3% up
           'day': 0
       },
       'step_2': {
           'ticker': 'TICKER2', 
           'condition': 'return > 0.02',  # 2% up
           'day': 1  # next day
       },
       'target': {
           'ticker': 'TICKER3',
           'measure': 'max_return',  # or 'end_return'
           'window': 3  # days to track
       }
   }
   ```

2. **Run all cells**

3. **Interpret results:**
   - **Occurrences > 10** = enough data
   - **Win rate > 60%** = edge confirmed
   - **Avg return > 3%** = tradeable

### Example Sequences to Test

**Uranium:**
```python
'step_1': {'ticker': 'UUUU', 'condition': 'return > 0.04', 'day': 0},
'step_2': {'ticker': 'NXE', 'condition': 'return > 0.03', 'day': 1},
'target': {'ticker': 'LEU', 'measure': 'max_return', 'window': 5}
```

**Quantum:**
```python
'step_1': {'ticker': 'IONQ', 'condition': 'return > 0.05', 'day': 0},
'step_2': {'ticker': 'RGTI', 'condition': 'return > 0.04', 'day': 0},  # same day
'target': {'ticker': 'QBTS', 'measure': 'max_return', 'window': 2}
```

🐺 **Hunt the pattern. Test the sequence. Trust the numbers.**